### Let's build GPT: from scratch, in code, spelled out.

In [64]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [65]:
with open("/Users/nihaljayanth/Development/makemore/data/input.txt", "r") as f:
    text = f.read()

In [66]:
len(text)

1115394

In [67]:
chars = sorted(set(list(text)))
vocab_size = len(chars)
print(f"Number of unique charactes in this dataset: {vocab_size},\nAnd they are: {''.join(chars)}")

Number of unique charactes in this dataset: 65,
And they are: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [68]:
stoi = {c:idx for idx, c in enumerate(chars)}
itos = {idx:c for idx, c in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [69]:
print(encode("Hii there"))
print(decode([20, 47, 47, 1, 58, 46, 43, 56, 43]))
print(decode(encode("Hii there")))

[20, 47, 47, 1, 58, 46, 43, 56, 43]
Hii there
Hii there


In [70]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)

torch.Size([1115394])


In [71]:
split = int(0.9 * len(data))
train_data = data[:split]
val_data = data[split:]

In [72]:
block_size = 8

In [73]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
    inp = x[:i+1]
    pred = y[i]
    print(f"when the input is {inp} the target is {pred}")

when the input is tensor([18]) the target is 47
when the input is tensor([18, 47]) the target is 56
when the input is tensor([18, 47, 56]) the target is 57
when the input is tensor([18, 47, 56, 57]) the target is 58
when the input is tensor([18, 47, 56, 57, 58]) the target is 1
when the input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when the input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [74]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else val_data
    sample = torch.randint(high=len(data)-block_size, size=(batch_size,))
    x = torch.stack([data[s:s+block_size] for s in sample])
    y = torch.stack([data[s+1:s+block_size+1] for s in sample])
    return x, y

In [75]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [76]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        # B = batch_size, T = seq_len, C = output_classes
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [77]:
lm = BigramLanguageModel(vocab_size=vocab_size)

In [78]:
logits, loss = lm(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [79]:
decode(lm.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=10)[0].tolist())

'\nSKIcLT;AcE'

In [80]:
optimizer = torch.optim.AdamW(lm.parameters(), lr=1e-3)

In [81]:
batch_size = 32

for steps in range(5000):
    xb, yb = get_batch("train")
    
    logits, loss = lm(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4737846851348877


In [82]:
for param in nn.Linear(2, 1).parameters():
    print(param)

Parameter containing:
tensor([[-0.0182,  0.0240]], requires_grad=True)
Parameter containing:
tensor([0.0692], requires_grad=True)


In [83]:
print(decode(lm.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


3Ak'ze ity sth wbf s thitowh HMan -- b, swillef r, knoutep my, ppand:

Comiwd ruy y
T; to:' w-! MI cie tund IOHASHEGSt takpceseARy
LElan$
LAdould, gCOHER:

IUT:
ThSSAgGO haimilangs :prndsd conNUDqVopxllay KI imee m. cillt halamngen.
K:CothJh tansFMByse d qKa By e ha; g thFlod ONRiny
Y: iMAGoci?$OKEncr cot d s atocy tin s t aqIO:
OF: lon, grs nt?
Susu3filldl lloreth---pe yad wor ilotarenoreehallyond ocal, o: inon o
MIstinwMasthZxthnerthe nkill.
HoxRI prnicn f beanksVCuay d tRThatate M; d bo honds


### Mathematical trick in self-attention

In [84]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [85]:
wei = torch.ones(size=(T,T))
wei = torch.tril(wei)
wei = wei / wei.sum(dim=1, keepdim=True)
xbow = wei @ x

In [86]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [87]:
tril = torch.tril(torch.ones(size=(T,T)))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [88]:
wei = wei.masked_fill(tril == 0, float("-inf"))
F.softmax(wei, dim=-1)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [89]:
tril = torch.tril(torch.ones(size=(T,T)))
wei = torch.zeros(size=(T,T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [106]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(size=(B, T, C))

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1)
wei = wei * head_size**-0.5

tril = torch.tril(torch.ones(size=(T, T)))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

In [109]:
tril[:T, :T]

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [110]:
ln = nn.LayerNorm(32)

In [ ]:
x = torch.randn(size=(1,1,32)) * 10

In [ ]:
ln(x).mean()

tensor(1.4901e-08, grad_fn=<MeanBackward0>)

In [117]:
0 % 0

ZeroDivisionError: integer modulo by zero

In [123]:
n_embed = 512

class Head(nn.Module):
    '''Single headed self attention'''

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        print(f"x shape: {x.shape}")
        B, T, C = x.shape
        k = self.key(x)
        print(f"k shape: {k.shape}")
        q = self.query(x)
        v = self.value(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        print(f"wei shape: {wei.shape}")
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        out = wei @ v
        print(f"out shape: {out.shape}")
        return out

In [124]:
h = Head(head_size=64)

In [125]:
x = torch.randn((4, 8, 512))
y = h.forward(x)

x shape: torch.Size([4, 8, 512])
k shape: torch.Size([4, 8, 64])
wei shape: torch.Size([4, 8, 8])
out shape: torch.Size([4, 8, 64])


In [128]:
x = torch.randn((4, 8, 100))
y = torch.randn((4, 8))
x.view(4*8, 100).shape
y.view(4*8).shape

torch.Size([32])

In [127]:
x.view(4*8).shape

RuntimeError: shape '[32]' is invalid for input of size 3200